In [1]:
import gym
import random
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras import backend as K
import numpy as np

In [2]:

def initiate_NN(learning_rate=1e-3):
    model = Sequential()
    model.add(Dense(64, input_dim=8, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(4, activation='linear'))
    model.compile(loss='mse', optimizer=Adam(lr=learning_rate))
    return model

def epsilon_action(epsilon):
    random_value = np.random.random()
    if random_value<=epsilon:
        action = env.action_space.sample()
    else:
        Y=online_model.predict(state.reshape(1,-1))
        action = np.argmax(Y[0])
    return action


In [3]:


gamma=0.99
epsilon_decay = 0.005
learning_rate =0.0005
max_e=1
min_e=0.0001
replay_batch_size=30
iterations=1000 

In [6]:
env = gym.make('LunarLander-v2')
state=env.reset()
online_model = initiate_NN(learning_rate)
memory=[]
episode_reward_rolling = []
episode_reward=[]

for episode in range(iterations):
    state=env.reset()
    current_episode_reward = 0
    done=False
    steps=0
    epsilon = min_e + (max_e - min_e)*np.exp(-epsilon_decay*episode)
    while not done:
        action = epsilon_action(epsilon)
        new_state, reward, done, info= env.step(action)
        steps+=1
        current_episode_reward +=reward
        memory.append([state,action,reward,new_state,done])
        state = new_state
        if len(memory)>=replay_batch_size:
            batch = random.sample(memory, replay_batch_size)
            batch = np.array(batch)
            non_terminal_samples = np.where(batch[:, 4] == False)
            y = np.copy(batch[:, 2])
            if len( non_terminal_samples[0]) > 0:
                #use online q values to select best action
                online_q = online_model.predict(np.vstack(batch[:, 3]))
                #use offline q values of best action as target 
                y[non_terminal_samples] += np.multiply(gamma, \
                            online_q[non_terminal_samples, \
                            np.argmax(online_q[non_terminal_samples, :][0], axis=1)][0])
            actions = np.array(batch[:, 1], dtype=int)
            y_target = online_model.predict(np.vstack(batch[:, 0]))
            y_target[range(replay_batch_size), actions] = y
            online_model.fit(np.vstack(batch[:, 0]), y_target, epochs=1, verbose=0)

    episode_reward.append(current_episode_reward) 
    if len(memory)>=5000:
        memory = memory[-5000:]
    if len(episode_reward)>=100:
        episode_reward_rolling.append(np.average(episode_reward[-100:]))
    else:
        episode_reward_rolling.append(np.average(episode_reward))
    if np.sum(np.array(episode_reward_rolling[-20:])>200)>19 and epsilon<0.1:
        break
    print('episode: ', episode, ' episode reward: ', '%.2f' % current_episode_reward, \
          ' rolling reward: ', '%.2f' % episode_reward_rolling[-1], ' Steps: ', steps, ' epsilon: ',\
          '%.2f' % epsilon)

episode:  0  episode reward:  -2.53  rolling reward:  -2.53  Steps:  106  epsilon:  1.00
episode:  1  episode reward:  -77.10  rolling reward:  -39.82  Steps:  79  epsilon:  1.00
episode:  2  episode reward:  -426.16  rolling reward:  -168.60  Steps:  88  epsilon:  1.00
episode:  3  episode reward:  -42.35  rolling reward:  -137.04  Steps:  75  epsilon:  0.99
episode:  4  episode reward:  -151.88  rolling reward:  -140.01  Steps:  95  epsilon:  0.99
episode:  5  episode reward:  -117.21  rolling reward:  -136.21  Steps:  79  epsilon:  0.99
episode:  6  episode reward:  -212.56  rolling reward:  -147.11  Steps:  91  epsilon:  0.99
episode:  7  episode reward:  -82.44  rolling reward:  -139.03  Steps:  62  epsilon:  0.99
episode:  8  episode reward:  -101.08  rolling reward:  -134.81  Steps:  122  epsilon:  0.98
episode:  9  episode reward:  -85.17  rolling reward:  -129.85  Steps:  71  epsilon:  0.98
episode:  10  episode reward:  -302.25  rolling reward:  -145.52  Steps:  113  epsilon:

episode:  89  episode reward:  -277.10  rolling reward:  -147.93  Steps:  112  epsilon:  0.84
episode:  90  episode reward:  -117.19  rolling reward:  -147.59  Steps:  124  epsilon:  0.84
episode:  91  episode reward:  -194.51  rolling reward:  -148.10  Steps:  115  epsilon:  0.83
episode:  92  episode reward:  14.38  rolling reward:  -146.35  Steps:  149  epsilon:  0.83
episode:  93  episode reward:  -144.34  rolling reward:  -146.33  Steps:  166  epsilon:  0.83
episode:  94  episode reward:  -149.36  rolling reward:  -146.36  Steps:  107  epsilon:  0.83
episode:  95  episode reward:  -71.31  rolling reward:  -145.58  Steps:  76  epsilon:  0.83
episode:  96  episode reward:  -65.83  rolling reward:  -144.76  Steps:  69  epsilon:  0.83
episode:  97  episode reward:  -108.05  rolling reward:  -144.38  Steps:  69  epsilon:  0.82
episode:  98  episode reward:  -135.95  rolling reward:  -144.30  Steps:  61  epsilon:  0.82
episode:  99  episode reward:  -113.28  rolling reward:  -143.99  St

episode:  177  episode reward:  -102.20  rolling reward:  -97.29  Steps:  105  epsilon:  0.70
episode:  178  episode reward:  -28.85  rolling reward:  -96.58  Steps:  113  epsilon:  0.70
episode:  179  episode reward:  -105.61  rolling reward:  -96.73  Steps:  155  epsilon:  0.70
episode:  180  episode reward:  -119.22  rolling reward:  -96.89  Steps:  120  epsilon:  0.70
episode:  181  episode reward:  -34.69  rolling reward:  -96.37  Steps:  113  epsilon:  0.70
episode:  182  episode reward:  -107.62  rolling reward:  -96.79  Steps:  127  epsilon:  0.69
episode:  183  episode reward:  -66.51  rolling reward:  -97.01  Steps:  87  epsilon:  0.69
episode:  184  episode reward:  -92.50  rolling reward:  -97.20  Steps:  158  epsilon:  0.69
episode:  185  episode reward:  -74.21  rolling reward:  -97.99  Steps:  126  epsilon:  0.69
episode:  186  episode reward:  -69.88  rolling reward:  -96.73  Steps:  130  epsilon:  0.69
episode:  187  episode reward:  67.07  rolling reward:  -94.52  Ste

episode:  266  episode reward:  -80.25  rolling reward:  -60.35  Steps:  120  epsilon:  0.59
episode:  267  episode reward:  -37.82  rolling reward:  -59.23  Steps:  143  epsilon:  0.59
episode:  268  episode reward:  -21.66  rolling reward:  -58.52  Steps:  139  epsilon:  0.59
episode:  269  episode reward:  -45.93  rolling reward:  -58.02  Steps:  110  epsilon:  0.58
episode:  270  episode reward:  -59.11  rolling reward:  -57.43  Steps:  151  epsilon:  0.58
episode:  271  episode reward:  -78.61  rolling reward:  -56.69  Steps:  133  epsilon:  0.58
episode:  272  episode reward:  -241.74  rolling reward:  -58.65  Steps:  238  epsilon:  0.58
episode:  273  episode reward:  9.06  rolling reward:  -57.35  Steps:  150  epsilon:  0.58
episode:  274  episode reward:  -27.86  rolling reward:  -56.67  Steps:  90  epsilon:  0.58
episode:  275  episode reward:  -318.51  rolling reward:  -59.71  Steps:  634  epsilon:  0.58
episode:  276  episode reward:  -18.59  rolling reward:  -59.10  Steps:

episode:  355  episode reward:  -412.04  rolling reward:  -50.60  Steps:  537  epsilon:  0.49
episode:  356  episode reward:  -46.18  rolling reward:  -50.65  Steps:  129  epsilon:  0.49
episode:  357  episode reward:  -359.00  rolling reward:  -53.48  Steps:  498  epsilon:  0.49
episode:  358  episode reward:  -1.63  rolling reward:  -53.16  Steps:  143  epsilon:  0.49
episode:  359  episode reward:  -241.02  rolling reward:  -55.56  Steps:  275  epsilon:  0.49
episode:  360  episode reward:  -200.61  rolling reward:  -55.11  Steps:  193  epsilon:  0.49
episode:  361  episode reward:  2.50  rolling reward:  -54.92  Steps:  197  epsilon:  0.49
episode:  362  episode reward:  30.54  rolling reward:  -54.95  Steps:  1000  epsilon:  0.48
episode:  363  episode reward:  43.63  rolling reward:  -54.22  Steps:  1000  epsilon:  0.48
episode:  364  episode reward:  -14.11  rolling reward:  -54.17  Steps:  1000  epsilon:  0.48
episode:  365  episode reward:  105.60  rolling reward:  -51.97  Ste

...

episode:  620  episode reward:  -171.28  rolling reward:  -29.01  Steps:  250  epsilon:  0.29
episode:  621  episode reward:  17.60  rolling reward:  -29.50  Steps:  103  epsilon:  0.29
episode:  622  episode reward:  -107.84  rolling reward:  -30.70  Steps:  104  epsilon:  0.29
episode:  623  episode reward:  122.91  rolling reward:  -29.02  Steps:  1000  epsilon:  0.29
episode:  624  episode reward:  134.97  rolling reward:  -27.87  Steps:  1000  epsilon:  0.29
episode:  625  episode reward:  -156.42  rolling reward:  -28.76  Steps:  231  epsilon:  0.29
episode:  626  episode reward:  -259.43  rolling reward:  -31.17  Steps:  802  epsilon:  0.29
episode:  627  episode reward:  97.46  rolling reward:  -29.71  Steps:  1000  epsilon:  0.29
episode:  628  episode reward:  159.35  rolling reward:  -26.48  Steps:  648  epsilon:  0.28
episode:  629  episode reward:  107.12  rolling reward:  -25.96  Steps:  1000  epsilon:  0.28
episode:  630  episode reward:  -46.73  rolling reward:  -26.03 

episode:  709  episode reward:  -14.42  rolling reward:  -3.02  Steps:  1000  epsilon:  0.24
episode:  710  episode reward:  96.92  rolling reward:  -1.21  Steps:  1000  epsilon:  0.24
episode:  711  episode reward:  -235.56  rolling reward:  -4.33  Steps:  119  epsilon:  0.24
episode:  712  episode reward:  -25.92  rolling reward:  -4.60  Steps:  148  epsilon:  0.24
episode:  713  episode reward:  -8.70  rolling reward:  -5.34  Steps:  267  epsilon:  0.24
episode:  714  episode reward:  -143.05  rolling reward:  -8.27  Steps:  130  epsilon:  0.24
episode:  715  episode reward:  -97.52  rolling reward:  -10.37  Steps:  351  epsilon:  0.24
episode:  716  episode reward:  -426.22  rolling reward:  -14.70  Steps:  235  epsilon:  0.24
episode:  717  episode reward:  -102.80  rolling reward:  -18.03  Steps:  373  epsilon:  0.24
episode:  718  episode reward:  -157.41  rolling reward:  -20.07  Steps:  317  epsilon:  0.24
episode:  719  episode reward:  -195.46  rolling reward:  -21.93  Steps

episode:  797  episode reward:  -451.53  rolling reward:  -46.06  Steps:  121  epsilon:  0.20
episode:  798  episode reward:  -34.22  rolling reward:  -43.66  Steps:  1000  epsilon:  0.20
episode:  799  episode reward:  56.83  rolling reward:  -44.13  Steps:  1000  epsilon:  0.20
episode:  800  episode reward:  78.81  rolling reward:  -43.41  Steps:  1000  epsilon:  0.20
episode:  801  episode reward:  -105.39  rolling reward:  -42.36  Steps:  465  epsilon:  0.20
episode:  802  episode reward:  -79.09  rolling reward:  -43.94  Steps:  1000  epsilon:  0.20
episode:  803  episode reward:  105.96  rolling reward:  -43.37  Steps:  1000  epsilon:  0.20
episode:  804  episode reward:  259.76  rolling reward:  -42.65  Steps:  852  epsilon:  0.20
episode:  805  episode reward:  282.60  rolling reward:  -39.56  Steps:  745  epsilon:  0.20
episode:  806  episode reward:  -131.31  rolling reward:  -42.09  Steps:  1000  epsilon:  0.20
episode:  807  episode reward:  -4.83  rolling reward:  -43.18 

episode:  885  episode reward:  231.26  rolling reward:  2.93  Steps:  536  epsilon:  0.17
episode:  886  episode reward:  -211.62  rolling reward:  3.00  Steps:  215  epsilon:  0.17
episode:  887  episode reward:  267.22  rolling reward:  5.15  Steps:  477  epsilon:  0.17
episode:  888  episode reward:  241.65  rolling reward:  6.88  Steps:  894  epsilon:  0.17
episode:  889  episode reward:  202.66  rolling reward:  8.53  Steps:  666  epsilon:  0.17
episode:  890  episode reward:  202.42  rolling reward:  12.47  Steps:  726  epsilon:  0.17
episode:  891  episode reward:  181.59  rolling reward:  15.10  Steps:  832  epsilon:  0.17
episode:  892  episode reward:  247.23  rolling reward:  17.02  Steps:  868  epsilon:  0.17
episode:  893  episode reward:  264.55  rolling reward:  21.07  Steps:  733  epsilon:  0.17
episode:  894  episode reward:  239.56  rolling reward:  23.67  Steps:  612  epsilon:  0.17
episode:  895  episode reward:  223.40  rolling reward:  27.35  Steps:  529  epsilon

episode:  974  episode reward:  30.76  rolling reward:  13.07  Steps:  250  epsilon:  0.14
episode:  975  episode reward:  227.41  rolling reward:  15.70  Steps:  572  epsilon:  0.14
episode:  976  episode reward:  65.59  rolling reward:  21.58  Steps:  173  epsilon:  0.14
episode:  977  episode reward:  266.72  rolling reward:  22.71  Steps:  441  epsilon:  0.14
episode:  978  episode reward:  287.65  rolling reward:  24.07  Steps:  524  epsilon:  0.14
episode:  979  episode reward:  165.80  rolling reward:  24.27  Steps:  833  epsilon:  0.14
episode:  980  episode reward:  -116.06  rolling reward:  21.13  Steps:  123  epsilon:  0.14
episode:  981  episode reward:  54.24  rolling reward:  19.70  Steps:  1000  epsilon:  0.14
episode:  982  episode reward:  225.52  rolling reward:  19.73  Steps:  688  epsilon:  0.14
episode:  983  episode reward:  240.30  rolling reward:  20.15  Steps:  738  epsilon:  0.14
episode:  984  episode reward:  -88.89  rolling reward:  17.90  Steps:  103  epsi

In [7]:

for episode in range(1000,1500):
    state=env.reset()
    current_episode_reward = 0
    done=False
    steps=0
    epsilon = min_e + (max_e - min_e)*np.exp(-epsilon_decay*episode)
    while not done:
        action = epsilon_action(epsilon)
        new_state, reward, done, info= env.step(action)
        steps+=1
        current_episode_reward +=reward
        memory.append([state,action,reward,new_state,done])
        state = new_state
        if len(memory)>=replay_batch_size:
            batch = random.sample(memory, replay_batch_size)
            batch = np.array(batch)
            non_terminal_samples = np.where(batch[:, 4] == False)
            y = np.copy(batch[:, 2])
            if len( non_terminal_samples[0]) > 0:
                #use online q values to select best action
                online_q = online_model.predict(np.vstack(batch[:, 3]))
                #use offline q values of best action as target 
                y[non_terminal_samples] += np.multiply(gamma, \
                            online_q[non_terminal_samples, \
                            np.argmax(online_q[non_terminal_samples, :][0], axis=1)][0])
            actions = np.array(batch[:, 1], dtype=int)
            y_target = online_model.predict(np.vstack(batch[:, 0]))
            y_target[range(replay_batch_size), actions] = y
            online_model.fit(np.vstack(batch[:, 0]), y_target, epochs=1, verbose=0)

    episode_reward.append(current_episode_reward) 
    if len(memory)>=5000:
        memory = memory[-5000:]
    if len(episode_reward)>=100:
        episode_reward_rolling.append(np.average(episode_reward[-100:]))
    else:
        episode_reward_rolling.append(np.average(episode_reward))
    if np.sum(np.array(episode_reward_rolling[-20:])>200)>19 and epsilon<0.1:
        break
    print('episode: ', episode, ' episode reward: ', '%.2f' % current_episode_reward, \
          ' rolling reward: ', '%.2f' % episode_reward_rolling[-1], ' Steps: ', steps, ' epsilon: ',\
          '%.2f' % epsilon)

episode:  1000  episode reward:  186.96  rolling reward:  1.08  Steps:  676  epsilon:  0.14
episode:  1001  episode reward:  204.98  rolling reward:  5.91  Steps:  984  epsilon:  0.14
episode:  1002  episode reward:  70.67  rolling reward:  4.98  Steps:  749  epsilon:  0.13
episode:  1003  episode reward:  163.63  rolling reward:  7.15  Steps:  585  epsilon:  0.13
episode:  1004  episode reward:  17.06  rolling reward:  5.08  Steps:  1000  epsilon:  0.13
episode:  1005  episode reward:  109.76  rolling reward:  11.17  Steps:  1000  epsilon:  0.13
episode:  1006  episode reward:  196.10  rolling reward:  13.96  Steps:  836  epsilon:  0.13
episode:  1007  episode reward:  248.45  rolling reward:  16.89  Steps:  527  epsilon:  0.13
episode:  1008  episode reward:  11.19  rolling reward:  21.51  Steps:  158  epsilon:  0.13
episode:  1009  episode reward:  -5.36  rolling reward:  30.48  Steps:  293  epsilon:  0.13
episode:  1010  episode reward:  -190.29  rolling reward:  32.49  Steps:  353

episode:  1088  episode reward:  236.27  rolling reward:  -105.96  Steps:  387  epsilon:  0.11
episode:  1089  episode reward:  -481.01  rolling reward:  -109.52  Steps:  115  epsilon:  0.11
episode:  1090  episode reward:  199.21  rolling reward:  -106.77  Steps:  322  epsilon:  0.11
episode:  1091  episode reward:  -426.92  rolling reward:  -107.89  Steps:  95  epsilon:  0.11
episode:  1092  episode reward:  254.20  rolling reward:  -107.64  Steps:  472  epsilon:  0.11
episode:  1093  episode reward:  237.98  rolling reward:  -107.87  Steps:  445  epsilon:  0.11
episode:  1094  episode reward:  -481.14  rolling reward:  -114.89  Steps:  85  epsilon:  0.11
episode:  1095  episode reward:  -322.77  rolling reward:  -120.38  Steps:  91  epsilon:  0.11
episode:  1096  episode reward:  -313.36  rolling reward:  -122.99  Steps:  108  epsilon:  0.11
episode:  1097  episode reward:  -237.69  rolling reward:  -125.47  Steps:  108  epsilon:  0.11
episode:  1098  episode reward:  -87.39  rollin

episode:  1174  episode reward:  -42.54  rolling reward:  -163.77  Steps:  1000  epsilon:  0.10
episode:  1175  episode reward:  -115.50  rolling reward:  -159.11  Steps:  1000  epsilon:  0.10
episode:  1176  episode reward:  -277.33  rolling reward:  -157.56  Steps:  155  epsilon:  0.10
episode:  1177  episode reward:  -118.82  rolling reward:  -152.71  Steps:  311  epsilon:  0.10
episode:  1178  episode reward:  -14.31  rolling reward:  -150.61  Steps:  1000  epsilon:  0.09
episode:  1179  episode reward:  -48.29  rolling reward:  -147.87  Steps:  1000  epsilon:  0.09
episode:  1180  episode reward:  -89.77  rolling reward:  -147.06  Steps:  1000  epsilon:  0.09
episode:  1181  episode reward:  -87.72  rolling reward:  -139.99  Steps:  1000  epsilon:  0.09
episode:  1182  episode reward:  -45.86  rolling reward:  -137.44  Steps:  1000  epsilon:  0.09


KeyboardInterrupt: 

In [ ]:
online_model.save_weights('weights_singleDQN_'+str(max_e)+'-'+str(min_e)+'-'+str(epsilon_decay)\
                         +'-'+str(gamma)+'-'+str(replay_batch_size)+'-'+str(learning_rate)+'.h5')

import pickle

with open('weights_singleDQN_'+str(max_e)+'-'+str(min_e)+'-'+str(epsilon_decay)\
                         +'-'+str(gamma)+'-'+str(replay_batch_size)+'-'+str(learning_rate)+'.json', 'wb') as fp:
    pickle.dump({'rewards':episode_reward,'rolling_reward':episode_reward_rolling}, fp)


In [167]:
rward = pickle.load( open( 'weights_DQN_'+str(max_e)+'-'+str(min_e)+'-'+str(epsilon_decay)\
                         +'-'+str(gamma)+'-'+str(replay_batch_size)+'-'+str(learning_rate)+'.json', "rb" ) )

In [168]:
rward

{'rewards': [-131.3223570990876, -248.23258991485326, -133.48636655895385],
 'rolling_reward': [-131.3223570990876,
  -189.77747350697044,
  -171.01377119096492]}

In [163]:
episode_reward

[]

In [152]:
Y_target.shape

(5, 4)

In [142]:
batch[terminal_samples].shape

(0, 5)

In [125]:



# i:index for sample [state,action,reward,new_state,done]
for i,sample in enumerate(batch):
    terminal_sample = sample[4]
    new_state_sample = sample[3]
    state_sample = sample[0]
    X[i]=state_sample
    action_sample = sample[1]
    Y_target[i] = online_model.predict(state_sample.reshape(1,-1))[0]
    if terminal:
        #terminal then targetY is reward
        Y_target[i][action_sample] = sample[2]
    else:
        #use online weights to select argmax action under new_state

        online_qvalues_batch =online_model.predict(new_state_sample.reshape(1,-1)) 
        max_action = np.argmax(online_qvalues_batch)
        #use target weights to calculate Qvalue
        #non-terminal then targetY is reward +gamma*new Qvalue
        Y_target[i][action_sample] = sample[2] + gamma * target_model.predict(new_state_sample.reshape(1,-1)).reshape(4,)[max_action]

online_model.fit(np.array(X), np.array(Y_target), verbose=0)


100

In [ ]:
Notice that the selection of the action, in the argmax, is


In [91]:
online_qvalues_batch

[array([[ 0.18740644, -0.2721934 ,  0.2233217 , -0.45678362]],
       dtype=float32),
 array([[ 0.19613695, -0.27545747,  0.20470577, -0.46681872]],
       dtype=float32),
 array([[ 0.18746991, -0.25133634,  0.19952038, -0.4359238 ]],
       dtype=float32),
 array([[ 0.19334945, -0.2700162 ,  0.19816679, -0.45707738]],
       dtype=float32),
 array([[ 0.19308917, -0.27386588,  0.21208042, -0.46272066]],
       dtype=float32)]

In [113]:
np.array(X)

array([[-0.00213823,  1.4003743 , -0.21659775, -0.4687014 ,  0.0024845 ,
         0.04906261,  0.        ,  0.        ],
       [-0.00846777,  1.3651963 , -0.20548455, -0.5481141 ,  0.00758275,
         0.00502167,  0.        ,  0.        ],
       [-0.01887293,  1.3004851 , -0.22156799, -0.61512053,  0.00625335,
         0.04479944,  0.        ,  0.        ],
       [-0.01250668,  1.3402183 , -0.20310256, -0.5355056 ,  0.00488083,
        -0.02924636,  0.        ,  0.        ],
       [-0.00641537,  1.3775289 , -0.21631236, -0.5210343 ,  0.00733169,
         0.04846335,  0.        ,  0.        ]], dtype=float32)

In [112]:
Y_target

[array([-1.2552428 , -0.2741652 ,  0.2263453 , -0.45686692], dtype=float32),
 array([ 0.19308917, -0.27386588,  0.21208042, -0.6722371 ], dtype=float32),
 array([ 0.1933487, -0.2546284,  5.1014786, -0.445849 ], dtype=float32),
 array([-0.91460276, -0.27277392,  0.20281965, -0.4571485 ], dtype=float32),
 array([ 0.18904   , -0.27001685,  0.21992686, -0.7534547 ], dtype=float32)]

In [ ]:

class DDQL:
    def __init__(self, nS, nA):
        self.nS = nS
        self.nA = nA
        self.epsilon = 1
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.9993
        self.gamma = 0.99
        self.learning_rate = 0.0001
        self.epochs = 1
        self.verbose = 0
        self.minibatch_size = 30
        self.memory = deque(maxlen=5000)
        self.model = self.create_model()
        self.target_model = self.create_model()

    def create_model(self):
        model = Sequential()

        # Add 2 hidden layers with 64 nodes each
        model.add(Dense(64, input_dim=self.nS, activation='relu'))
        model.add(Dense(64, activation='relu'))
        model.add(Dense(self.nA, activation='linear'))
        model.compile(loss='mse', optimizer=Adam(lr=self.learning_rate))
        return model

    def add_memory(self, s, a, r, s_prime, done):
        self.memory.append((s, a, r, s_prime, done))

    def target_model_update(self):
        self.target_model.set_weights(self.model.get_weights())

    def selectAction(self, s):
        if np.random.rand() <= self.epsilon:
            return np.random.choice(self.nA)
        q = self.model.predict(s)
        return np.argmax(q[0])

    def replay(self):
        # Vectorized method for experience replay
        minibatch = random.sample(self.memory, self.minibatch_size)
        minibatch = np.array(minibatch)
        not_done_indices = np.where(minibatch[:, 4] == False)
        y = np.copy(minibatch[:, 2])

        # If minibatch contains any non-terminal states, use separate update rule for those states
        if len(not_done_indices[0]) > 0:
            predict_sprime = self.model.predict(np.vstack(minibatch[:, 3]))
            predict_sprime_target = self.target_model.predict(np.vstack(minibatch[:, 3]))
            
            # Non-terminal update rule
            y[not_done_indices] += np.multiply(self.gamma, \
                        predict_sprime_target[not_done_indices, \
                        np.argmax(predict_sprime[not_done_indices, :][0], axis=1)][0])

        actions = np.array(minibatch[:, 1], dtype=int)
        y_target = self.model.predict(np.vstack(minibatch[:, 0]))
        y_target[range(self.minibatch_size), actions] = y
        self.model.fit(np.vstack(minibatch[:, 0]), y_target, epochs=self.epochs, verbose=self.verbose)

    def replayIterative(self):
        # Iterative method - this performs the same function as replay() but is not vectorized 
        s_list = []
        y_state_list = []
        minibatch = random.sample(self.memory, self.minibatch_size)
        for s, a, r, s_prime, done in minibatch:
            s_list.append(s)
            y_action = r
            if not done:
                y_action = r + self.gamma * np.amax(self.model.predict(s_prime)[0])

            print(y_action)
            
            y_state = self.model.predict(s)
            y_state[0][a] = y_action
            y_state_list.append(y_state)
        self.model.fit(np.squeeze(s_list), np.squeeze(y_state_list), batch_size=self.minibatch_size, epochs=1, verbose=0)

if __name__ == "__main__":
    np.set_printoptions(precision=2)
    env = gym.make('LunarLander-v2')
    env = wrappers.Monitor(env, tdir, force=True, video_callable=False)

    nS = env.observation_space.shape[0]
    nA = env.action_space.n

    agent = DDQL(nS, nA)

    # Set to true to use saved model
    episodes = 10000

    # Cumulative reward
    reward_avg = deque(maxlen=100)

    for e in range(episodes):
        episode_reward = 0
        s = env.reset()
        s = np.reshape(s, [1, nS])

        for time in range(1000):

            # Query next action from learner and perform action
            a = agent.selectAction(s)
            s_prime, r, done, info = env.step(a)

            # Add cumulative reward
            episode_reward += r

            # Reshape new state
            s_prime = np.reshape(s_prime, [1, nS])

            # Add experience to memory
            agent.add_memory(s, a, r, s_prime, done)

            # Set current state to new state
            s = s_prime

            #Perform experience replay if memory length is greater than minibatch length
   
            if len(agent.memory) > agent.minibatch_size:
                agent.replay()

            # If episode is done, exit loop
            if done:
                agent.target_model_update()
                break


        # epsilon decay
        if agent.epsilon > agent.epsilon_min:
            agent.epsilon *= agent.epsilon_decay

        # Running average of past 100 episodes
        reward_avg.append(episode_reward)
        print('episode: ', e, ' score: ', '%.2f' % episode_reward, ' avg_score: ', '%.2f' % np.average(reward_avg), ' frames: ', time, ' epsilon: ', '%.2f' % agent.epsilon)
        
        # with open('trained_agent.txt', 'a') as f:
        #     f.write(str(np.average(reward_avg)) + '\n')

    env.close()
#     gym.upload(tdir, api_key='sk_EJo79Jo0RsSEA4EF6Cp5mg')


#agent.model.save_weights('weights.h5')
#agent.model.load_weights('weights.h5')

episode:  0  score:  -101.20  avg_score:  -101.20  frames:  104  epsilon:  1.00
episode:  1  score:  -147.99  avg_score:  -124.59  frames:  74  epsilon:  1.00
episode:  2  score:  -95.67  avg_score:  -114.95  frames:  74  epsilon:  1.00
episode:  3  score:  -218.30  avg_score:  -140.79  frames:  94  epsilon:  1.00
episode:  4  score:  -231.98  avg_score:  -159.03  frames:  86  epsilon:  1.00
episode:  5  score:  -161.32  avg_score:  -159.41  frames:  68  epsilon:  1.00
episode:  6  score:  -96.64  avg_score:  -150.44  frames:  66  epsilon:  1.00
episode:  7  score:  -141.43  avg_score:  -149.32  frames:  97  epsilon:  0.99
episode:  8  score:  -480.81  avg_score:  -186.15  frames:  102  epsilon:  0.99
episode:  9  score:  -371.18  avg_score:  -204.65  frames:  92  epsilon:  0.99
episode:  10  score:  -161.56  avg_score:  -200.73  frames:  95  epsilon:  0.99
episode:  11  score:  -80.20  avg_score:  -190.69  frames:  61  epsilon:  0.99
episode:  12  score:  -332.47  avg_score:  -201.60 

episode:  103  score:  -168.96  avg_score:  -179.65  frames:  118  epsilon:  0.93
episode:  104  score:  -64.43  avg_score:  -177.97  frames:  91  epsilon:  0.93
episode:  105  score:  -200.70  avg_score:  -178.36  frames:  72  epsilon:  0.93
episode:  106  score:  32.45  avg_score:  -177.07  frames:  121  epsilon:  0.93
episode:  107  score:  -260.25  avg_score:  -178.26  frames:  117  epsilon:  0.93
episode:  108  score:  -110.97  avg_score:  -174.56  frames:  80  epsilon:  0.93
episode:  109  score:  -253.15  avg_score:  -173.38  frames:  94  epsilon:  0.93
episode:  110  score:  -125.12  avg_score:  -173.02  frames:  88  epsilon:  0.93
episode:  111  score:  -189.74  avg_score:  -174.11  frames:  72  epsilon:  0.92
episode:  112  score:  -140.07  avg_score:  -172.19  frames:  94  epsilon:  0.92
episode:  113  score:  -304.20  avg_score:  -173.17  frames:  71  epsilon:  0.92
episode:  114  score:  -128.94  avg_score:  -173.02  frames:  72  epsilon:  0.92
episode:  115  score:  -294.

episode:  204  score:  -298.51  avg_score:  -150.87  frames:  97  epsilon:  0.87
episode:  205  score:  -77.35  avg_score:  -149.64  frames:  146  epsilon:  0.87
episode:  206  score:  -124.22  avg_score:  -151.21  frames:  68  epsilon:  0.87
episode:  207  score:  -40.81  avg_score:  -149.01  frames:  144  epsilon:  0.86
episode:  208  score:  -83.08  avg_score:  -148.73  frames:  67  epsilon:  0.86
episode:  209  score:  -268.33  avg_score:  -148.89  frames:  92  epsilon:  0.86
episode:  210  score:  -124.07  avg_score:  -148.88  frames:  105  epsilon:  0.86
episode:  211  score:  -123.48  avg_score:  -148.21  frames:  122  epsilon:  0.86
episode:  212  score:  -143.46  avg_score:  -148.25  frames:  80  epsilon:  0.86
episode:  213  score:  -86.07  avg_score:  -146.07  frames:  96  epsilon:  0.86
episode:  214  score:  -86.35  avg_score:  -145.64  frames:  94  epsilon:  0.86
episode:  215  score:  -55.86  avg_score:  -143.25  frames:  93  epsilon:  0.86
episode:  216  score:  -58.61 